In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go
from ipywidgets import widgets
from ipywidgets.embed import embed_minimal_html
import warnings
warnings.filterwarnings("ignore")

In [2]:
user_quests = pd.read_csv('/Users/hanpu/Documents/Genki/final_user_quest.csv')

In [3]:
ga = pd.read_csv('google analytics - Pages and screens.csv')

In [4]:
ga_new = ga[ga['网页标题和屏幕名称'].isin(user_quests['campaignTitle'])].reset_index(drop=True)
ga_new

,网页标题和屏幕名称,查看次数,用户数,新用户数,每位用户的浏览量,平均互动时长,唯一身份用户滚动次数,事件数,转化次数,总收益
0,GenkiBox Beta Test NFT Giveaway,12587,6237,6175,2.018118,52.846080,4469,46585,0,0
1,Rh!noX NFT discount & U Giveaway,8270,1639,1540,5.045760,95.885906,1290,25751,0,0
2,GENKI TWITTER SPACE X WOWSWAP X BETAMARS,8168,3439,2810,2.375109,60.403606,2876,31624,0,0
3,GENKI TWITTER SPACE X ONTO WALLET X SIL FINANCE,7642,2899,1765,2.636081,86.454295,2492,28676,0,0
4,ZEBRA X VERA MYSTERY BOX GIVEAWAY,6500,2722,1720,2.387950,92.100294,2288,25454,0,0
5,The Project Galaxy & RhizomeDAO AMA Giveaway,3762,1520,1043,2.475000,64.277632,1329,13349,0,0
6,BABYSWAP X GENKI AMA CAMPAIGN,1502,629,555,2.387917,46.879173,521,5376,0,0
7,WOWmax Airdrop Program,1354,374,172,3.620321,59.074866,312,4256,0,0
8,Genki ✖️ BabyWealthyClub 20 NFT Whitelists Giv...,1091,578,275,1.887543,66.399654,497,4122,0,0
9,GenkiBox Demo Campaign,1089,463,304,2.352052,52.408207,391,3876,0,0


In [5]:
user_quests['quesType'].unique()

array(['web2/discord/joinserver', 'web2/twitter/retweet',
       'web2/twitter/follow', 'web3', 'referral', 'web2/telegram/join',
       'web2/link', 'web2/answer', 'web2/twitter/like_and_retweet'],
      dtype=object)

# get the quest number in every campaign

In [6]:
# get the quest number in every campaign
quest_num = user_quests.groupby(['campaignTitle','questId','quesType']).count().reset_index()
quest_num['new_type'] = 'web2'
for i in range(len(quest_num)):
  if quest_num['quesType'].iloc[i] == 'web3':
    quest_num['new_type'].iloc[i] = 'web3'
  elif quest_num['quesType'].iloc[i] == 'referral':
    quest_num['new_type'].iloc[i] = 'referral'

In [7]:
quest_num = quest_num.groupby(['campaignTitle','new_type']).count()['questId'].reset_index()

In [8]:
quest_num.columns = ['campaignTitle', 'new_type', 'number']

In [9]:
quest_num[(quest_num['campaignTitle'] == 'Apecoinflip Bounty Program') & (quest_num['new_type'] == 'referral')]

,campaignTitle,new_type,number
2,Apecoinflip Bounty Program,referral,1


# a middle dataframe to connect the two df

In [10]:
# and aggregate the campaign and quest type
# referral is not included
ga_mid = user_quests[user_quests['quesType'] != 'referral']

ga_mid['new_type'] = 'web2'
for i in range(len(ga_mid)):
  if ga_mid['quesType'].iloc[i] == 'web3':
    ga_mid['new_type'].iloc[i] = 'web3'


In [11]:
ga_mid = ga_mid.groupby(['campaignTitle','userAddress','new_type']).count()['questId'].reset_index()

In [12]:
ga_mid.columns = ['campaignTitle', 'userAddress', 'new_type', 'count']

In [13]:
ga_mid.columns.name=None

In [14]:
filtered_ga = pd.DataFrame()
for i in range(len(ga_mid)):
  # 2 filter to locate the number of quest
  filter1 = (quest_num['campaignTitle'] == ga_mid['campaignTitle'].iloc[i])
  filter2 = (quest_num['new_type'] == ga_mid['new_type'].iloc[i])
  # get the total quest
  total_quest = quest_num[filter1 & filter2]['number'].iloc[0]
  # only select those equals to total_quest
  if ga_mid['count'].iloc[i] == total_quest:
    filtered_ga = filtered_ga.append(ga_mid.iloc[i])

In [15]:
filtered_ga

,campaignTitle,userAddress,new_type,count
0,AirNFTs x BabyWealthyClub 20 NFT Whitelists Gi...,0x02ba49001d5ecc223f0a53131cb3d53106387c2e,web2,6
2,AirNFTs x BabyWealthyClub 20 NFT Whitelists Gi...,0x05688259a7b50c0f92a8db7cd4f97ebe965c5cb3,web2,6
5,AirNFTs x BabyWealthyClub 20 NFT Whitelists Gi...,0x123092dd76973aa075a318e78e4bf8a91c9b82ed,web2,6
6,AirNFTs x BabyWealthyClub 20 NFT Whitelists Gi...,0x131f07a63cc7a6b31f7527cb118d9c576da9adfc,web2,6
8,AirNFTs x BabyWealthyClub 20 NFT Whitelists Gi...,0x171cc136a959c864038c7dcb1ad48e54b2c06dbd,web2,6
...,...,...,...,...
8529,twitterSpace,0xfa2a56c6cfa68deea34f2640b259036fe824a2e9,web2,8
8531,twitterSpace,0xfbd08fd2b2c7148b4e38624158c3155b5b49f886,web2,8
8534,twitterSpace,0xfd0beee9d7be139b9a24fff8316a5c15760776de,web2,8
8535,twitterSpace,0xfe14943cc76b5c4013de35acb7f8ec4183bd7f1c,web2,8


In [16]:
filtered_ga = filtered_ga.groupby(['campaignTitle','new_type']).sum().reset_index()
filtered_ga = filtered_ga.pivot(index='campaignTitle',columns='new_type')['count'].reset_index()


In [17]:
ga_final = pd.merge(ga,filtered_ga[['campaignTitle','web2','web3']],how='inner',left_on='网页标题和屏幕名称',right_on='campaignTitle')

In [18]:
ga_final

,网页标题和屏幕名称,查看次数,用户数,新用户数,每位用户的浏览量,平均互动时长,唯一身份用户滚动次数,事件数,转化次数,总收益,campaignTitle,web2,web3
0,GenkiBox Beta Test NFT Giveaway,12587,6237,6175,2.018118,52.846080,4469,46585,0,0,GenkiBox Beta Test NFT Giveaway,2475.0,1360.0
1,Rh!noX NFT discount & U Giveaway,8270,1639,1540,5.045760,95.885906,1290,25751,0,0,Rh!noX NFT discount & U Giveaway,216.0,36.0
2,GENKI TWITTER SPACE X WOWSWAP X BETAMARS,8168,3439,2810,2.375109,60.403606,2876,31624,0,0,GENKI TWITTER SPACE X WOWSWAP X BETAMARS,6125.0,NaN
3,GENKI TWITTER SPACE X ONTO WALLET X SIL FINANCE,7642,2899,1765,2.636081,86.454295,2492,28676,0,0,GENKI TWITTER SPACE X ONTO WALLET X SIL FINANCE,3801.0,NaN
4,ZEBRA X VERA MYSTERY BOX GIVEAWAY,6500,2722,1720,2.387950,92.100294,2288,25454,0,0,ZEBRA X VERA MYSTERY BOX GIVEAWAY,36.0,NaN
5,The Project Galaxy & RhizomeDAO AMA Giveaway,3762,1520,1043,2.475000,64.277632,1329,13349,0,0,The Project Galaxy & RhizomeDAO AMA Giveaway,3625.0,NaN
6,BABYSWAP X GENKI AMA CAMPAIGN,1502,629,555,2.387917,46.879173,521,5376,0,0,BABYSWAP X GENKI AMA CAMPAIGN,498.0,7.0
7,WOWmax Airdrop Program,1354,374,172,3.620321,59.074866,312,4256,0,0,WOWmax Airdrop Program,72.0,NaN
8,Genki ✖️ BabyWealthyClub 20 NFT Whitelists Giv...,1091,578,275,1.887543,66.399654,497,4122,0,0,Genki ✖️ BabyWealthyClub 20 NFT Whitelists Giv...,1495.0,NaN
9,GenkiBox Demo Campaign,1089,463,304,2.352052,52.408207,391,3876,0,0,GenkiBox Demo Campaign,3.0,4.0


In [19]:
ga_final_1 = ga_final[['campaignTitle','查看次数','web2','web3']]
ga_final_1.columns = ['campaignTitle','View count','web2 count','web3 count']

In [20]:
# do this to avoid error when exporting html
ga_final_1['View count'] = ga_final_1['View count'].astype('float')

In [33]:
ga_final_1

,campaignTitle,View count,web2 count,web3 count
0,GenkiBox Beta Test NFT Giveaway,12587.0,2475.0,1360.0
1,Rh!noX NFT discount & U Giveaway,8270.0,216.0,36.0
2,GENKI TWITTER SPACE X WOWSWAP X BETAMARS,8168.0,6125.0,NaN
3,GENKI TWITTER SPACE X ONTO WALLET X SIL FINANCE,7642.0,3801.0,NaN
4,ZEBRA X VERA MYSTERY BOX GIVEAWAY,6500.0,36.0,NaN
5,The Project Galaxy & RhizomeDAO AMA Giveaway,3762.0,3625.0,NaN
6,BABYSWAP X GENKI AMA CAMPAIGN,1502.0,498.0,7.0
7,WOWmax Airdrop Program,1354.0,72.0,NaN
8,Genki ✖️ BabyWealthyClub 20 NFT Whitelists Giv...,1091.0,1495.0,NaN
9,GenkiBox Demo Campaign,1089.0,3.0,4.0


In [22]:
campaignName = widgets.Dropdown(
    options=list(ga_final_1['campaignTitle']),
    value='GenkiBox Beta Test NFT Giveaway'
)

In [23]:
figure = go.Figure(go.Funnel(
    y = ga_final_1.columns[1:],
    x = ga_final_1.iloc[0][1:])
  )
  
figure.show()

In [24]:
g = go.FigureWidget(data=figure,
                    layout=go.Layout(
                        title=dict(
                            text='NYC FlightDatabase'
                        )
                    ))

In [25]:
# Let now write a function that will handle the input from the widgets, and alter the state of the graph.


def response(change):
  temp_df = ga_final_1[ga_final_1['campaignTitle'] == campaignName.value]

  x_ = temp_df.iloc[0][1:]
  with g.batch_update():
      g.data[0].x = x_
      g.layout.xaxis.title = 'Steps'
      g.layout.title =  'Funnel of campaign: ' + campaignName.value

campaignName.observe(response, names="value")


In [26]:
container = widgets.HBox([widgets.Label(value="Campaign name: "),campaignName])


final_fig = widgets.VBox([container,
              g])

In [27]:
final_fig

In [28]:
embed_minimal_html('funnel.html', views=[final_fig], title='Widgets export')

In [29]:
type(final_fig)

ipywidgets.widgets.widget_box.VBox

In [38]:
final_fig

In [44]:
fig = go.Figure()


fig = go.Figure()

fig.add_trace(go.Funnelarea(values = [500, 450, 340, 230, 220, 110], textinfo = "value",
    title = {"position": "top center", "text": "Sales for Sale Person A in U.S."},
    domain = {"x": [0, 0.5], "y": [0, 0.5]}))

fig.add_trace(go.Funnelarea(values = [600, 500, 400, 300, 200, 100], textinfo = "value",
    title = {"position": "top center", "text": "Sales of Sale Person B in Canada"},
    domain = {"x": [0, 0.5], "y": [0.55, 1]}))


fig.show(

)